<a href="https://colab.research.google.com/github/OsmanTurgut2335/covid-data-project/blob/main/AttackDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandas openpyxl


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
import matplotlib.pyplot as plt

# Excel dosyasından veri setini yükle
df = pd.read_excel('db.xlsx')

# Veri setinin ilk birkaç satırını görüntüle
print(df.head())


In [ ]:
# Enerji (energy) hesaplama
df['energy_mWh'] = df['current_mA'] * df['bus_voltage_V']

print(df.head())


In [ ]:
# One-hot encoding işlemi
df_encoded = pd.get_dummies(df, columns=['State', 'Attack'])

# Kodlanmış veri setinin ilk birkaç satırını görüntüle
print(df_encoded.head())


In [ ]:
# Özellikleri (X) ve hedef değişkeni (y) ayır
X = df_encoded[['shunt_voltage','bus_voltage_V', 'current_mA', 'power_mW','State_charging', 'State_idle']]
y = df_encoded[['Attack_Backdoor', 'Attack_cryptojacking', 'Attack_none',
    'Attack_syn-flood', 'Attack_syn-stealth', 'Attack_tcp-flood', 'Attack_vuln-scan']]

# Normalleştirme için min-max ölçekleyici veya standart ölçekleyici kullanılabilir
# Ölçekleyiciyi başlat
scaler = StandardScaler()

# Özellikleri normalleştir
X_normalized = scaler.fit_transform(X)

# Veri kümesini eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(
    X_normalized, y, test_size=0.3, random_state=42)

In [ ]:
from tensorflow.keras.layers import Dropout
model = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(7, activation='softmax')
])


# Modeli derle
model.compile(optimizer='adam',   #optimizasyon algoritması
              loss='categorical_crossentropy',  #kayıp fonksiyonu
              metrics=['accuracy'])


# Modeli eğit
history =model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.3)  # Gerektiğinde epoch ve toplu boyut sayısını ayarlayabilirsiniz

# Test seti üzerinde modeli değerlendir
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", test_accuracy)
# Get the weights of the trained model
layer_weights = model.get_weights()

# Extract the weights of the first layer (input layer to the first hidden layer)
input_layer_weights = layer_weights[0]

# Calculate the absolute sum of weights for each feature
feature_importance = np.sum(np.abs(input_layer_weights), axis=1)

# Normalize feature importance scores
feature_importance /= np.sum(feature_importance)

# Match feature importance scores with feature names
feature_names = X.columns

# Create a dictionary to store feature importance scores with feature names
feature_importance_dict = dict(zip(feature_names, feature_importance))


# Sort feature importance scores in descending order
sorted_feature_importance = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

# Extract feature names and importance scores
features, importance = zip(*sorted_feature_importance)

# Plotting feature importance
plt.figure(figsize=(10, 6))
plt.barh(range(len(features)), importance, align='center')
plt.yticks(range(len(features)), features)
plt.xlabel('Normalized Importance')
plt.title('Feature Importance')
plt.show()

In [ ]:
# Eğitim ve doğrulama kaybını çiz
plt.plot(history.history['loss'], label='Eğitim Kaybı')
plt.plot(history.history['val_loss'], label='Doğrulama Kaybı')
plt.xlabel('Epoch')
plt.ylabel('Kayıp')
plt.legend()
plt.show()


In [ ]:
import pandas as pd
import numpy as np

# Excel dosyasından veri setini yükle
external_data_df = pd.read_excel('simulation_logs.xlsx')

# Veri setinin ilk birkaç satırını görüntüle
print(external_data_df.head())

# One-hot encoding işlemi için 'State' sütununu ekleyin
external_data_encoded = pd.get_dummies(external_data_df, columns=['State'])

# One-hot encoding yapıldıktan sonra veri kümesini görüntüleyin
print(external_data_encoded.head())




In [ ]:
# One-hot encoding yapılan veri kümesinden gerekli özellikleri seçin
X_external_encoded = external_data_encoded[['shunt_voltage', 'bus_voltage_V', 'current_mA', 'power_mW', 'State_charging', 'State_idle']]

# Verileri ölçekleyin
X_external_normalized = scaler.transform(X_external_encoded)

# Modeli kullanarak tahmin yapın
predictions = model.predict(X_external_normalized)

# Tahmin olasılıklarını yazdırın (her bir sınıf için olasılık)
print(predictions)

# En yüksek olasılığa sahip olan sınıfı seçin
predicted_classes = np.argmax(predictions, axis=1)

# Tahmin edilen sınıfları yazdırın
print(predicted_classes)

class_names = ['Attack_Backdoor', 'Attack_cryptojacking', 'Attack_none', 'Attack_syn-flood', 'Attack_syn-stealth', 'Attack_tcp-flood', 'Attack_vuln-scan']

predicted_class_names = [class_names[idx] for idx in predicted_classes]

print(predicted_class_names)


In [ ]:
# Tahmin edilen sınıfları sütun olarak ekleyin
external_data_df['Predicted_Attack'] = predicted_class_names

# Veri setinin güncellenmiş halini gözden geçirin
print(external_data_df.head())

# Güncellenmiş veri setini Excel dosyasına kaydedin
external_data_df.to_excel('tahmin_edilen.xlsx', index=False)

print("Tahmin edilen sınıflar Excel dosyasına başarıyla eklendi.")


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Random Forest modelini oluşturun
model = RandomForestClassifier()

# Modeli eğitin
model.fit(X_train, y_train)

# Modelin performansını değerlendirin
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Model Accuracy:", accuracy)


Model Accuracy: 0.6282201405152225


In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import precision_score, recall_score



def preprocessing(filename):
  """
  Excel dosyasını okur, temizler ve istatistiğini döndürür
  :param filename: .xlsx formatındaki dosya path'i
  """
  df = pd.read_excel(filename)
  df.dropna(inplace=True)
  print("Columns:\n", df.columns)
  print("\nInfo:\n", df.info())
  print("\nDescription:\n", df.describe())
  print("\nUnique values of the column 'Attack':\n", df['Attack'].unique())
  print("\nUnique values of the column 'State':\n", df['State'].unique())
  return df


def training(df):
  """
  Modelin Representation aşamasıdır, encoding, scaling ve training yapılır
  :param df: Temizlenmiş DataFrame
  """
  # Encoding ve değişken ayırma işlemi
  encoded_df = pd.get_dummies(df, columns=['State']) # Bağımsız değişkenlere
  X = encoded_df.drop('Attack', axis=1)
  y = encoded_df['Attack']

  # Scaling işlemi, StandardScaler burada uygun olmayabilir
  scaler = MinMaxScaler(feature_range=(0, 10))
  X_scaled = scaler.fit_transform(X)

  # Train-Test kümeleri ayrışımı
  X_train, X_test, y_train, y_test = train_test_split(
      X_scaled, y, test_size=0.2, random_state=42
  )

  # Model eğitimi, çok sınıflı Lojistik Regresyon algoritması kullanılır
  model = LogisticRegression(
      multi_class='multinomial', solver='lbfgs', max_iter=1000
  )
  model.fit(X_train, y_train)

  return model, X_test, y_test


def evaluation(model, X_test, y_test):
  """
  Eğitilen model ile tahminler yapılır ve sonuçlar incelenir
  :param model: Eğitilmiş model
  :param X_test: Test edilecek veri, bağımlı değişken yok
  :param y_test: Hedef sütunun gerçek değerleri
  """
  y_pred = model.predict(X_test)
  print("\nAccuracy değeri:", accuracy_score(y_test, y_pred))
  print("\nClassification Report:\n", classification_report(y_test, y_pred))
  print("\nPrecision Score:", precision_score(y_test, y_pred, average='weighted'))
  print("\nRecall Score:", recall_score(y_test, y_pred, average='weighted'), '\n')


def main():
  filename = "db.xlsx"
  data = preprocessing(filename)
  trained_model, X_test, y_test = training(data)
  evaluation(trained_model, X_test, y_test)


if __name__ == '__main__':
  main()
